In [1]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.autograd import Variable

import numpy as np

In [2]:
class c3d():
    pass

In [3]:
class SST_AD_Prop(nn.Module):
    """Implementation of SSTAD proposal module, for use in full SSTAD model.
    Forwards both the SST output proposals and the corresponding hidden state representation.
    """
    def __init__(self, num_proposals=16, num_rnn_layers=1, h_width=512, input_size=500, dropout=0):
        super(SST_AD_Prop, self).__init__() 
        self.rnn = nn.GRU(input_size=input_size, hidden_size=h_width, num_layers=num_rnn_layers, dropout=dropout)
        self.lin_out = nn.Linear(h_width, num_proposals)#512-->16
        self.nonlin_final = nn.Sigmoid()
        self.num_rnn_layers = num_rnn_layers
        self.h_width = h_width

    def init_hidden_state(self, batch_sz):
        return Variable(torch.zeros(self.num_rnn_layers, batch_sz, self.h_width))

    def forward(self, inputs):
        batch_sz = inputs.size(0) # should be batch_sz (~200 in old set-up)
        inputs = torch.transpose(inputs,0,1)
        h0 = self.init_hidden_state(batch_sz)
        rnn_output, h_n = self.rnn.forward(inputs, h0)
       
        # get proposals output (L x N x h_width) ==> (N x L x K)
        output = self.lin_out.forward(rnn_output.view(rnn_output.size(0)*rnn_output.size(1), rnn_output.size(2)))#(len_seq,batch_size,512)-->(len_seq,batch_size,16)
        
        lin_out = output.view(rnn_output.size(0), rnn_output.size(1), output.size(1))
        final_out = self.nonlin_final(torch.transpose(lin_out,0,1))
        return final_out, rnn_output

    #new edit,原始代码的rnn的output和hidden_size的形状一样大概是？脑子不好？
    def m_prop_loss(self, h_s_r_p, y_k, w0_k,w1_k):
        '''
        代码
        h_s_r_p: hidden state representation of prop. should be shape of [sequence_len, batch_sz, num_poposals]
        y_k: should be shape of [sequence_len, batch_sz, num_poposals] , ground truth from data for all scales of segments
        w0_k: shape of [sequence_len, batch_sz, num_poposals]. constant value for a given video in all time step t.
            for a given scale k,w0_k = num_of_positive/all
        w1_k: shape of [sequence_len, batch_sz, num_poposals]. constant value for a given video.
            for a given scale k,w0_k = num_of_negtive/all  

        '''
        #m_mrop = self.nonlin_final( self.lin_out(h_t) )
        m_mrop = h_s_r_p
        return -torch.sum((torch.log(m_mrop)*y_k*w0_k + torch.log(1-m_mrop)*(1-y_k)*w1_k))
        

In [4]:
final_out, rnn_output = gru_prop(Variable(torch.randn(10,6,500)))

NameError: name 'gru_prop' is not defined

In [5]:
type(final_out)

NameError: name 'final_out' is not defined

## y-k是labei？
w1，w2是该poposoal为正样本占所有的长度

In [6]:
y_k = Variable(torch.randn(10,6,16))
w1 = Variable(torch.randn(10,6,16))
w2 = Variable(torch.randn(10,6,16))

In [7]:
gru_prop.m_prop_loss(final_out,y_k, w1, w2)

NameError: name 'gru_prop' is not defined

In [8]:
class SST_AD_SegAct(nn.Module):
    """Implementation of SSTAD segment action classification module, for use in full SSTAD model.
    Forwards both the output segment classification and the corresponding hidden state representation.
    ** Note that num_classes = number of classes + 1 for background class.
    """
    def __init__(self, num_classes=201, num_rnn_layers=1, h_width=512, input_size=500, dropout=0, init_range=None,**kwargs):
        super(SST_AD_SegAct, self).__init__() 
        self.rnn = nn.GRU(input_size=input_size, hidden_size=h_width, num_layers=num_rnn_layers, dropout=dropout) #, batch_first=True)
        self.lin_out = nn.Linear(h_width, num_classes)
        self.nonlin_eval = torch.nn.Softmax()
        self.num_rnn_layers = num_rnn_layers
        self.h_width = h_width
        self.init_weights(init_range)

    def init_weights(self, init_range):
        if init_range is None:
            return
        elif isinstance(init_range, float):
            init_range = (init_range,) * 2
        elif len(init_range) != 2:
            raise ValueError('Argument ({}) different than tuple with size {} '
                             'different than 2'.format(type(init_range),
                                                       len(init_range)))

        for i in range(self.num_rnn_layers):
            weight_var = getattr(self.rnn, 'weight_ih_l{}'.format(i))
            weight_var.data.uniform_(-init_range[0], init_range[0])
        self.lin_output.weight.data.uniform_(-init_range[1], init_range[1])
        self.lin_out.bias.data.fill_(0)

    def init_hidden_state(self, batch_sz):
        return Variable(torch.zeros(self.num_rnn_layers, batch_sz, self.h_width))

    def forward(self, inputs):
        batch_sz = inputs.size(0) # should be batch_sz (~200 in old set-up)
        inputs = torch.transpose(inputs,0,1)
        h0 = self.init_hidden_state(batch_sz)
        rnn_output, h_n = self.rnn.forward(inputs, h0)
        # get "output" after linear layer. 
        output = self.lin_out.forward(rnn_output.view(rnn_output.size(0)*rnn_output.size(1), rnn_output.size(2)))
        L, N = rnn_output.size(0), rnn_output.size(1)
        C = output.size(1)
        assert L*N == output.size(0), "ERROR: mismatch in output tensor dimensions"
        fin_out = output.view(L, N, C) 
        fin_out = torch.transpose(fin_out,0,1) 
        fin_out = fin_out.contiguous().view(N*L, C)
        return fin_out, rnn_output

    def obtain_class_scores(self, output, rnn_output_size, flatten_two_dim=True): 
        # use this during evaluation of segment-level classification.
        L, N = rnn_output_size[0], rnn_output_size[1]
        C = output.size(1)
        assert L*N == output.size(0), "ERROR: mismatch in output tensor dimensions"
        class_scores = self.nonlin_eval(output)
        if flatten_two_dim:
            return class_scores
        return class_scores.view(N, L, C)
    
    
    def m_cls_loss(self, h_s_r_c, y_t, wt):
        '''
        h_s_r_c: should be shape of [batch_sz * sequence_len, num_class], top hidden state on the stacked of GRUs at time step t
        y_t: should be shape of [batch_sz * sequence_len, num_class], ground truth from data for a segments of classes
        wt: a constant number if y(t) indicates the segment is background, and 1 otherwise
        '''
        m_cls = self.nonlin_eval( h_s_r_c )#(batch_sz, num_prop)
        return torch.sum(-wt*torch.log(torch.sum(m_cls*y_t,-1)))
        

In [9]:
gru_prop = SST_AD_Prop()
gru_cls = SST_AD_SegAct()

In [10]:
final_out, rnn_output = gru_cls(Variable(torch.randn(10,6,500)))

In [11]:
final_out.shape

torch.Size([60, 201])

### y_t= Variable(torch.randn(60,201))#对应的是seqence_len10*batch_sz6, 201个classes
wt = Variable(torch.randn(60))#seqence_len10*batch_sz6个数据段是否是背景

In [12]:
gru_cls.m_cls_loss(final_out, y_t, wt)

NameError: name 'y_t' is not defined

In [13]:
class SST_AD_PreRel(nn.Module): # Note: basic pre-release version... will be updating code for actual release. 
    """Note: basic pre-release factored version of SS-TAD. Stay tuned for updated versions coming later :)
    """
    def __init__(self, num_proposals=16, num_classes=21, num_rnn_layers=1, h_width=512, input_size=500, dropout=0):
        super().__init__() 
        self.SST_Prop = SST_AD_Prop(num_proposals, num_rnn_layers, h_width, input_size, dropout)
        self.SST_SegAct = SST_AD_SegAct(num_classes, num_rnn_layers, h_width, input_size, dropout)
        self.num_rnn_layers = num_rnn_layers
        self.num_proposals = num_proposals
        self.num_classes = num_classes
        self.eval_non_lin = nn.Softmax()
        self.lin_out = nn.Linear(h_width*2, num_proposals * num_classes)
        self.h_width = h_width

    def forward(self, inputs):
        (proposals_out, h_prop), (class_out, h_cls) = \
                (self.SST_Prop.forward(inputs), self.SST_SegAct.forward(inputs))
        h_comb = torch.cat((h_prop,h_cls), 2) 
        self.h_cls_size = h_cls.size()
        self.seq_len = h_comb.size(0)
        self.batch_num = h_comb.size(1)
        output = self.lin_out.forward(h_comb.view(h_comb.size(0)*h_comb.size(1), h_comb.size(2)))
        output = self.convert_to_batch_order(output, self.batch_num, self.seq_len, self.num_proposals, self.num_classes)
        final_joint_cls_scores = self.get_class_scores(output)
        return output, proposals_out, class_out, final_joint_cls_scores

    def get_seg_class_scores(self, class_out):
        return self.SST_SegAct.obtain_class_scores(class_out, self.h_cls_size)

    def convert_to_batch_order(self, output, N, L, K, C):
        output = output.view(L, N, K, C)
        output = torch.transpose(output, 0,1)
        return output.contiguous().view(N*L*K, C)

    def get_class_scores(self, output): 
        class_scores = self.eval_non_lin(output)
        fin_out = class_scores.view(self.batch_num, self.seq_len, self.num_proposals, self.num_classes) 
        return fin_out 
    
    def loss(v_k, y_k, over_lap_vct, indicator_vct,):
        '''
        y_k,o_k都是和训练数据相关的，还没处理
        '''
        
        l_detcls = -torch.sum(w_k*torch.log(torch.sum(v_k*y_k)),dim = 2)
        l_detloc = (1/2)* torch.sum( ( torch.pow(torch.sum(v_k*y_k),2)/over_lap_vct -1 )*indicator_vct,dim = 2)
        l_det = l_detcls + lamda_detloc*l_detloc
        l = torch.sum(lamda_prop*l_prop + lamda_cls*l_cls + lamda_det*l_det)
        return l

In [14]:
gru_prop

SST_AD_Prop(
  (rnn): GRU(500, 512)
  (lin_out): Linear(in_features=512, out_features=16)
  (nonlin_final): Sigmoid()
)

## for test time code

#### import numpy as np

def get_segments(y, delta=16):
    """Convert predicted output tensor (y_pred) from SST model into the
    corresponding temporal proposals. Can perform standard confidence
    thresholding/post-processing (e.g. non-maximum suppression) to select
    the top proposals afterwards.

    Parameters
    ----------
    y : ndarray
        Predicted output from SST model of size (L, K), where L is the length of
        the input video in terms of discrete time steps.
    delta : int, optional
        The temporal resolution of the visual encoder in terms of frames. See
        Section 3 of the main paper for additional details.

    Returns
    -------
    props : ndarray
        Two-dimensional array of shape (num_props, 2), containing the start and
        end boundaries of the temporal proposals in units of frames.
    scores : ndarray
        One-dimensional array of shape (num_props,), containing the
        corresponding scores for each detection above.
    """
    temp_props, temp_scores = [], []
    L, K = y.shape
    for i in range(L):
        for j in range(min(i+1, K)):
            temp_props.append([delta*(i-j-1), delta*i])
            temp_scores.append(y[i, j])
    props_arr, score_arr = np.array(temp_props), np.array(temp_scores)
    # filter out proposals that extend beyond the start of the video.
    idx_valid = props_arr[:, 0] >= 0
    props, scores = props_arr[idx_valid, :], score_arr[idx_valid]
    return props, scores


def nms_detections(props, scores, overlap=0.7):
    """Non-maximum suppression: Greedily select high-scoring detections and
    skip detections that are significantly covered by a previously selected
    detection. This version is translated from Matlab code by Tomasz
    Malisiewicz, who sped up Pedro Felzenszwalb's code.

    Parameters
    ----------
    props : ndarray
        Two-dimensional array of shape (num_props, 2), containing the start and
        end boundaries of the temporal proposals.
    scores : ndarray
        One-dimensional array of shape (num_props,), containing the corresponding
        scores for each detection above.

    Returns
    -------
    nms_props, nms_scores : ndarrays
        Arrays with the same number of dimensions as the original input, but
        with only the proposals selected after non-maximum suppression.
    """
    t1 = props[:, 0]
    t2 = props[:, 1]
    ind = np.argsort(scores)
    area = (t2 - t1 + 1).astype(float)
    pick = []
    while len(ind) > 0:
        i = ind[-1]
        pick.append(i)
        ind = ind[:-1]
        tt1 = np.maximum(t1[i], t1[ind])
        tt2 = np.minimum(t2[i], t2[ind])
        wh = np.maximum(0., tt2 - tt1 + 1.0)
        o = wh / (area[i] + area[ind] - wh)
        ind = ind[np.nonzero(o <= overlap)[0]]
    nms_props, nms_scores = props[pick, :], scores[pick]
    return nms_props, nms_scores

## y-k是labei？
w1，w2是该poposoal为正样本占所有的长度

# activity c3d

In [18]:
import json

In [ ]:
label_file = ''

In [51]:
    def timestamp_to_featstamp(self, timestamp, nfeats, duration):
        """
        Function to measure 1D overlap
        Convert the timestamps to feature indices
        """
        start, end = timestamp
        start = min(int(round(start / duration * nfeats)), nfeats - 1)
        end = max(int(round(end / duration * nfeats)), start + 1)
        return start, end

In [15]:
label_dataset = h5py.File(args.labels, 'w')

NameError: name 'h5py' is not defined

In [19]:
data = json.load(open(r'C:\Users\hasee\jupyter_wkd\sst\SST-master\SST-master\data\ActivityNet\activity_net.v1-3.min.json'))

In [20]:
data

{'database': {'sJFgo9H6zNo': {'annotations': [{'label': 'Fun sliding down',
     'segment': [0.01, 123.42336739937599]}],
   'duration': 139.042,
   'resolution': '640x360',
   'subset': 'training',
   'url': 'https://www.youtube.com/watch?v=sJFgo9H6zNo'},
  'V1zhqaGFY2A': {'annotations': [],
   'duration': 136.98,
   'resolution': '658x480',
   'subset': 'testing',
   'url': 'https://www.youtube.com/watch?v=V1zhqaGFY2A'},
  'JDg--pjY5gg': {'annotations': [{'label': 'Beer pong',
     'segment': [61.420442338881465, 64.56014561469672]}],
   'duration': 126.178,
   'resolution': '1920x1080',
   'subset': 'validation',
   'url': 'https://www.youtube.com/watch?v=JDg--pjY5gg'},
  'KsFid_YVsn0': {'annotations': [],
   'duration': 235.13,
   'resolution': '1920x1080',
   'subset': 'testing',
   'url': 'https://www.youtube.com/watch?v=KsFid_YVsn0'},
  '-TmWR_keSfI': {'annotations': [],
   'duration': 104.955,
   'resolution': '1280x720',
   'subset': 'testing',
   'url': 'https://www.youtube.c

In [21]:
data.keys()

dict_keys(['taxonomy', 'version', 'database'])

In [24]:
len(data['taxonomy'])

272

In [26]:
video_ids = list(data['database'].keys())

In [41]:
video_id=video_ids[5]

In [39]:
data['database'][video_ids[8000]]['annotations']

[{'label': 'Baton twirling',
  'segment': [0.010000000000000002, 156.35301897017894]}]

In [43]:
data['database'][video_id]['annotations']

[{'label': 'Swinging at the playground',
  'segment': [127.33201508846798, 201.39247572684465]}]

In [45]:
duration = data['database'][video_id]['duration']

In [46]:
duration

208.864

In [47]:
annotations = data['database'][video_id]['annotations']

In [48]:
annotations

[{'label': 'Swinging at the playground',
  'segment': [127.33201508846798, 201.39247572684465]}]

In [49]:
timestamps = [ann['segment'] for ann in annotations]

In [50]:
timestamps

[[127.33201508846798, 201.39247572684465]]

In [55]:
featstamps = [timestamp_to_featstamp(1 ,x, 100, duration) for x in timestamps]

In [56]:
featstamps

[(61, 96)]

In [58]:
data['taxonomy']

[{'nodeId': 389,
  'nodeName': 'Applying sunscreen',
  'parentId': 269,
  'parentName': 'Health-related self care'},
 {'nodeId': 269,
  'nodeName': 'Health-related self care',
  'parentId': 270,
  'parentName': 'Health-related Self Care'},
 {'nodeId': 270,
  'nodeName': 'Health-related Self Care',
  'parentId': 2,
  'parentName': 'Personal Care'},
 {'nodeId': 271,
  'nodeName': 'Arm wrestling',
  'parentId': 272,
  'parentName': 'Wrestling'},
 {'nodeId': 272,
  'nodeName': 'Wrestling',
  'parentId': 17,
  'parentName': 'Participating in Sports, Exercise, or Recreation'},
 {'nodeId': 273,
  'nodeName': 'Assembling bicycle',
  'parentId': 56,
  'parentName': 'Vehicle repair and maintenance (by self)'},
 {'nodeId': 274, 'nodeName': 'BMX', 'parentId': 275, 'parentName': 'Biking'},
 {'nodeId': 275,
  'nodeName': 'Biking',
  'parentId': 17,
  'parentName': 'Participating in Sports, Exercise, or Recreation'},
 {'nodeId': 276,
  'nodeName': 'Baking cookies',
  'parentId': 31,
  'parentName': '

In [59]:
len(data['taxonomy'])

272

In [84]:
data['taxonomy'][271]

{'nodeId': 268,
 'nodeName': 'Playing badminton',
 'parentId': 54,
 'parentName': 'Playing racquet sports '}

In [129]:
data['database'][video_id]['annotations'][0]['label']

'Swinging at the playground'

In [64]:
import h5py

In [65]:
label_dataset = h5py.File('22222222222222222222222222', 'w')

In [67]:
video_dataset = label_dataset.create_dataset(video_id, (100, 16), dtype='f')

In [80]:
video_dataset[...]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [85]:
par = []
for _ in data['taxonomy']:
    if _ ['nodeId'] not in par:
        par.append(_ ['nodeId'])
print(len(par))

272


In [105]:
aa = [i for i in range(300)]

# y_t

In [114]:
already_appear_list = []
look_up_dict_emmbeding={}
count = 0
for _ in data['taxonomy']:
    if _ ['nodeName'] not in already_appear_list:
        already_appear_list.append(_ ['nodeName'])
        look_up_dict_emmbeding[_ ['nodeName']] = count
        count+=1
print(count)

272


In [116]:
look_up_dict_emmbeding

272

In [107]:
data['taxonomy']

[{'nodeId': 389,
  'nodeName': 'Applying sunscreen',
  'parentId': 269,
  'parentName': 'Health-related self care'},
 {'nodeId': 269,
  'nodeName': 'Health-related self care',
  'parentId': 270,
  'parentName': 'Health-related Self Care'},
 {'nodeId': 270,
  'nodeName': 'Health-related Self Care',
  'parentId': 2,
  'parentName': 'Personal Care'},
 {'nodeId': 271,
  'nodeName': 'Arm wrestling',
  'parentId': 272,
  'parentName': 'Wrestling'},
 {'nodeId': 272,
  'nodeName': 'Wrestling',
  'parentId': 17,
  'parentName': 'Participating in Sports, Exercise, or Recreation'},
 {'nodeId': 273,
  'nodeName': 'Assembling bicycle',
  'parentId': 56,
  'parentName': 'Vehicle repair and maintenance (by self)'},
 {'nodeId': 274, 'nodeName': 'BMX', 'parentId': 275, 'parentName': 'Biking'},
 {'nodeId': 275,
  'nodeName': 'Biking',
  'parentId': 17,
  'parentName': 'Participating in Sports, Exercise, or Recreation'},
 {'nodeId': 276,
  'nodeName': 'Baking cookies',
  'parentId': 31,
  'parentName': '

In [86]:
'annotations': [{'label': 'Swinging at the playground',
   'segment': [127.33201508846798, 201.39247572684465]}]

SyntaxError: illegal target for annotation (<ipython-input-86-428e1ee3e912>, line 1)

In [118]:
np.zeros(len(look_up_dict_emmbeding)+1)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [120]:
aa = np.zeros((3,5))

In [122]:
aa[:,-1]=1

In [124]:
aa[0]  = [1,1,1,1,1]

In [125]:
aa

array([[ 1.,  1.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.]])